In [42]:
from openpyxl import load_workbook
from html.parser import HTMLParser
import requests,base64,os
from xml.etree import ElementTree  as ET
# 运行之前记得关闭DevSidecar或者别的代理工具


# 从1、GetLisRequest 接口（获取标本信息）
def get_patient_info(hospSampleID):
    data = f'''<?xml version="1.0" encoding="utf-8"?>
            <soap12:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap12="http://www.w3.org/2003/05/soap-envelope">
            <soap12:Body><GetLisRequest xmlns="http://tempuri.org/"><hospSampleID>{hospSampleID}</hospSampleID>
            </GetLisRequest></soap12:Body></soap12:Envelope>'''
    encode_data = data.encode('utf-8')
    # print(data)
    headers = {"Host": "10.10.11.196",
            "Content-Type": "application/soap+xml; charset=UTF-8",
            "Content-Length": str(len(encode_data)),
            "SOAPAction": "http://tempuri.org/GetLisRequest"}
    patient_info = requests.post('http://58.62.17.237:4431/ExtReportService', data=encode_data ,headers=headers)

    print('patient_info.header----')
    print(patient_info.headers)
    print('patient_info.text----') 
    print(patient_info.text)

    return patient_info.text


class Parse(HTMLParser):
    def __init__(self):
    #Since Python 3, we need to call the __init__() function 
    #of the parent class
        super().__init__()
        self.reset()
        self.data = None
 
    def handle_data(self, data):
        # print("Here's the data: ", data)
        print('HTMLParser parsering data ')
        self.data = data

 
 # 解析1GetLisRequest返回的xml信息：
def get_LisRequest_data(patient_info):
    def get_tag_raw(tag_name):
        return root[0].find(tag_name).text    
    dct = {}
    parser = Parse()
    parser.feed(patient_info)
    data = parser.data
    root = ET.fromstring(data)
    
    dct['lis_Barcode'] = get_tag_raw('lis_Barcode')
    dct['blood_time'] = get_tag_raw('blood_time') 
    dct['lis_combine_code'] = root[0].find('LisItems').find('lis_item_code').text
        
    all_sub_items = root[0].find('LisItems').findall('SubItems') 
    dct['all_lis_item_code'] =  [item.find('lis_subitem_code').text for item in all_sub_items]
    dct['all_lis_item_name'] =  [item.find('lis_subitem_name').text for item in all_sub_items]    
       
    print('----patient_info-all_lis_item_name dct -----') 
    print(dct)
    # print(all_lis_item_name)
    # with open('xmlfile/'+dct['lis_Barcode'] +'.xml','w',encoding='utf-8') as fh:
    #     fh.write(data)
    # return lis_Barcode,blood_time,lis_combine_code,all_lis_item_code
    return dct
   

# 解析Excel报告的信息,这里的flag有两个，分别是5和17，
# 如果是5，那就是 “雄激素5项(爱湾)”，如果是17，那就代表“17α-羟孕酮”
def get_excel_info(infile,hospSampleID,flag = '5'):
    all_lis_item_code =  ['32378', '32379', '32380', '32381', '32383']
    patient_info = get_patient_info(hospSampleID)
    dct = get_LisRequest_data(patient_info)
    ex  = load_workbook(infile)
    tb = ex['报告']
    lis_Barcode= tb.cell(row=4,column=3).value
    if flag == '5':
        dct['lis_Barcode']   = lis_Barcode.split('+')[0]
        
    if flag == '17':
        dct['lis_Barcode']   = lis_Barcode.split('+')[1]
    # 采样日期：	
    dct['blood_time'] = tb.cell(row=6,column=3).value
    dct['ext_receive_time'],dct['ext_check_time'],dct['ext_report_time'] =  tb.cell(row=7,column=3).value, \
                                                                            tb.cell(row=7,column=7).value, \
                                                                            tb.cell(row=7,column=10).value
   # sheet里面从B10到H16,即检测结果的内容：
    result_info = ''
    result_dct = {}
    i  = 0
    all_lis_item_code =  ['32378', '32379', '32380', '32383', '32381','32382']
    all_lis_item_name =  ['睾酮', '雄烯二酮', '硫酸脱氢表雄酮', '双氢睾酮', '脱氢表雄酮','17α-羟孕酮']
    all_lis_combine_code = ['32025']*5 + ['32024']
    for row in range(10,16):
        i += 1
        if flag == '5' and i ==6:
            continue 
        if flag == '17' and i  in (1,2,3,4,5):
            continue 
#         result_dct['ext_item_code'] = all_lis_item_code
        result_dct['ext_item_code'] = all_lis_item_code[i-1]
        result_dct['lis_item_name'] = all_lis_item_name[i-1]
        # ext_item_name = tb.cell(row=row,column=2).value
        result_dct['lis_item_code'] = all_lis_item_code[i-1] 
        result_dct['lis_combine_code'] = all_lis_combine_code[i-1]  

        result_dct['result'] = tb.cell(row=row,column=7).value   
        print('lis_item_nam--------')
        print(all_lis_item_name[i-1],all_lis_item_code[i-1] ,result_dct['result'] )
        # lis_item_name =  tb.cell(row=row,column=2).value
        result_dct['ext_combine_code'] = result_dct['lis_combine_code']
        
        result_info += set_result_info(result_dct)
    result_info = result_info + '<report_pic><pic_content>'  

    return result_info ,  dct



# 对每个检测项目的xml的result_info设置值
# 对每个检测项目的xml的result_info设置值
def set_result_info(dct):
    print('*'*20)
    print(dct)
    result_info = f'''    
    <result_info>
        <result_seq/>
        <ext_combine_code>{dct.get('ext_combine_code')}</ext_combine_code>
        <ext_combine_name>{dct.get('ext_combine_name')}</ext_combine_name>
        <ext_item_code>{dct.get('ext_item_code')}</ext_item_code>
        <ext_item_name>{dct.get('ext_item_name')}</ext_item_name>
        <result>        <![CDATA[{dct.get('result')}]]]]>>        <![CDATA[</result>
        <result_unit>        <![CDATA[{dct.get('result_unit')}]]]]>>   <![CDATA[</result_unit>
        <result_flag/>
        <result_reference>    <![CDATA[{dct.get('result_reference')}]]]]>>       <![CDATA[</result_reference>
        <result_date/>
        <result_intstrmt_name/>
        <result_test_method/>
        <result_suggestion/>
        <result_remark/>
        <lis_combine_code>{dct.get('lis_combine_code')}</lis_combine_code>
        <lis_combine_name>{dct.get('lis_combine_name')}</lis_combine_name>
        <lis_item_code>{dct.get('lis_item_code')}</lis_item_code>
        <lis_item_name>{dct.get('lis_item_name')}</lis_item_name>    
    </result_info>
   '''
    return result_info

# 要发送的xml的第一部分信息：
# import string 
def set_data1(dct):
    data1 = f'''
    <soap:Envelope
        xmlns:soap="http://www.w3.org/2003/05/soap-envelope"
        xmlns:tem="http://tempuri.org/">
        <soap:Header/>
        <soap:Body>
            <tem:UploadLisRepData>
                <!--Optional:-->
                <tem:reportResult>
                    <![CDATA[<Report_Result><Report_Info>
                    <ext_lab_code></ext_lab_code>
                    <lis_Barcode>{dct.get('lis_Barcode')}</lis_Barcode>
                    <ext_Barcode>{dct.get('ext_Barcode')}</ext_Barcode>
                    <ext_checkItem/>
                    <pat_name/>
                    <pat_age></pat_age>
                    <pat_height/>
                    <pat_wight/>
                    <pat_pre_week/>
                    <pat_id/>
                    <pat_bedNo/>
                    <pat_tel/>
                    <pat_sex/>
                    <pat_birthday/>
                    <pat_ori_name/>
                    <sam_name/>
                    <sam_state/>
                    <doctor_name/>
                    <dept_name/>
                    <clinical_diag/>
                    <SampleNumber/>
                    <blood_time>{dct.get('blood_time')}</blood_time>
                    <ext_check_ID/>
                    <ext_receive_time>{dct.get('ext_receive_time')}</ext_receive_time>
                    <ext_upload_time/>
                    <ext_report_suggestion/>
                    <ext_report_remark/>
                    <ext_intstrmt_name/>
                    <ext_lab_name/>
                    <ext_report_type/>
                    <ext_check_time/>
                    <ext_first_audit_time/>
                    <ext_first_audit_time/>
                    <ext_second_audit_time/>
                    <ext_checker/>
                    <ext_first_audit/>
                    <ext_second_audit/>
                    <ext_report_code>{dct.get('lis_Barcode')}</ext_report_code>
    '''
    return data1 


# pdf部分
def get_pdf(inpdf):
    data2 = ''
    with open(inpdf,'rb') as fh:
            data2 = base64.b64encode(fh.read())
    return data2

# 最后是将xml和pdf的信息发送到妇幼
def send_data(dct,data1,result_info,data2):
    # xml的结尾部分
    data3 = '''</pic_content><pic_name>{dct.get('lis_Barcode')}_report_base64.pdf</pic_name><pic_seq></pic_seq></report_pic></Report_Info></Report_Result>]]>
            </tem:reportResult>
        </tem:UploadLisRepData>
    </soap:Body>
</soap:Envelope>
    '''
    encode_data = data1.encode('utf-8') +result_info.encode('utf-8') + data2 +  data3.encode('utf-8')
    
    headers = {"Host": "10.10.11.196",
            "Content-Type": "text/xml; charset=utf-8",
            "Content-Length": str(len(encode_data)) ,#}
            "SOAPAction": "http://tempuri.org/UploadLisRepData"}
    patient_info = requests.post(f'http://58.62.17.237:4431/ExtReportService', data=encode_data ,headers=headers)

    print('patient_info.text----')    
    print(patient_info.text)

    if not os.path.exists('workdir'):
        os.mkdir('workdir')
    with open('./workdir/'+ dct.get('lis_Barcode') + '.xml','wb') as fh:
        fh.write(encode_data)




In [32]:
def main():
    # hospSampleID = '100311224212'
    hospSampleID = '3243146536'

    # 1 读取：从1、GetLisRequest接口（获取标本信息）
    patient_info = get_patient_info(hospSampleID)
    lis_Barcode,blood_time,lis_combine_code,all_lis_item_code = get_LisRequest_data(patient_info)
    # 2 读取Excel：
    word_infile = 'E:/project/common/2022/05/5.26/省妇幼pcos报告-2022-05-25/省妇幼pcos报告-2022-05-25/何丽敏-激素检测报告-2022-05-25.xlsx'
    # get_excel_info(word_infile,hospSampleID)

# main()

In [45]:
# hospSampleID = '100311224212'

# 1 读取：从1、GetLisRequest接口（获取标本信息）
# patient_info = get_patient_info(hospSampleID)
# dct = get_LisRequest_data(patient_info)
# 2 读取Excel：
# word_infile = 'E:/project/common/2022/05/5.26/省妇幼pcos报告-2022-05-25/省妇幼pcos报告-2022-05-25/何丽敏-激素检测报告-2022-05-25.xlsx'


patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 14:29:25 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '2300', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;4043146697&lt;/lis_Barcode&gt;&lt;sample_no&gt;3&lt;/sample_no&gt;&lt;pat_no&gt;2001155124&lt;/pat_no&gt;&lt;pat_id&gt;2001155124&lt;/pat_id&gt;&lt;pat_name&gt;何丽敏&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-20 08:45:47&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1998-03-14 00:00:00&lt;/pat_birthday&gt;&lt;pat

In [25]:
# print(result_info)

    
    <result_info>
        <result_seq/>
        <ext_combine_code>32025</ext_combine_code>
        <ext_combine_name>None</ext_combine_name>
        <ext_item_code>32378</ext_item_code>
        <ext_item_name>None</ext_item_name>
        <result>        <![CDATA[0.456]]]]>>        <![CDATA[</result>
        <result_unit>        <![CDATA[None]]]]>>   <![CDATA[</result_unit>
        <result_flag/>
        <result_reference>    <![CDATA[None]]]]>>       <![CDATA[</result_reference>
        <result_date/>
        <result_intstrmt_name/>
        <result_test_method/>
        <result_suggestion/>
        <result_remark/>
        <lis_combine_code>32025</lis_combine_code>
        <lis_combine_name>None</lis_combine_name>
        <lis_item_code>32378</lis_item_code>
        <lis_item_name>睾酮</lis_item_name>    
    </result_info>
       
    <result_info>
        <result_seq/>
        <ext_combine_code>32025</ext_combine_code>
        <ext_combine_name>None</ext_combine_name>
        <ext

In [49]:
hospSampleID = '3243146536'
infile = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/陈嘉绮-激素检测报告-2022-05-25.xlsx'
result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
# result_info , dct = get_excel_info(infile,hospSampleID,flag='17')

data1 = set_data1(dct)
inpdf =  'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/陈嘉绮-激素检测报告-2022-05-25.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 14:41:48 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '2300', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;3243146536&lt;/lis_Barcode&gt;&lt;sample_no&gt;9&lt;/sample_no&gt;&lt;pat_no&gt;2001916719&lt;/pat_no&gt;&lt;pat_id&gt;2001916719&lt;/pat_id&gt;&lt;pat_name&gt;陈嘉绮&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-20 08:32:40&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1999-10-28 00:00:00&lt;/pat_birthday&gt;&lt;pat

In [50]:
hospSampleID = '3343146537'
infile = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/陈嘉绮-激素检测报告-2022-05-25.xlsx'
# result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
result_info , dct = get_excel_info(infile,hospSampleID,flag='17')

data1 = set_data1(dct)
inpdf =  'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/陈嘉绮-激素检测报告-2022-05-25.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 14:42:10 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1706', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;3343146537&lt;/lis_Barcode&gt;&lt;sample_no&gt;10&lt;/sample_no&gt;&lt;pat_no&gt;2001916719&lt;/pat_no&gt;&lt;pat_id&gt;2001916719&lt;/pat_id&gt;&lt;pat_name&gt;陈嘉绮&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-20 08:32:59&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1999-10-28 00:00:00&lt;/pat_birthday&gt;&lt;pa

In [51]:
hospSampleID = '2643164512'
infile = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/陈钰莹-激素检测报告-2022-05-25.xlsx'
# result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
result_info , dct = get_excel_info(infile,hospSampleID,flag='5')

data1 = set_data1(dct)
inpdf =  'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/陈钰莹-激素检测报告-2022-05-25.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 14:43:30 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '2302', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;2643164512&lt;/lis_Barcode&gt;&lt;sample_no&gt;19&lt;/sample_no&gt;&lt;pat_no&gt;2001915824&lt;/pat_no&gt;&lt;pat_id&gt;2001915824&lt;/pat_id&gt;&lt;pat_name&gt;陈钰莹&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-22 08:07:35&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;2004-01-29 00:00:00&lt;/pat_birthday&gt;&lt;pa

In [52]:
hospSampleID = '2743164513'
infile = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/陈钰莹-激素检测报告-2022-05-25.xlsx'
# result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
result_info , dct = get_excel_info(infile,hospSampleID,flag='17')

data1 = set_data1(dct)
inpdf =  'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/陈钰莹-激素检测报告-2022-05-25.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 14:43:41 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1706', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;2743164513&lt;/lis_Barcode&gt;&lt;sample_no&gt;20&lt;/sample_no&gt;&lt;pat_no&gt;2001915824&lt;/pat_no&gt;&lt;pat_id&gt;2001915824&lt;/pat_id&gt;&lt;pat_name&gt;陈钰莹&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-22 08:07:19&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;2004-01-29 00:00:00&lt;/pat_birthday&gt;&lt;pa

In [ ]:
# print(result_info ,lis_Barcode, blood_time, ext_receive_time, ext_check_time, ext_report_time)

lst_code = ["2343127051","4043146697","3143146922","2843125193","3243146536","2743136442","2243137022","2943156181","3243177523","2643164512","3343165806","2843173172"]
# lst_code = ["2343127051"]

def get_xml(lst_code):
    for hospSampleID in lst_code:
        patient_info = get_patient_info(hospSampleID)
        get_LisRequest_data(patient_info)
        
# get_xml(lst_code)    

In [17]:
print('2')

2


In [53]:
hospSampleID = '2343127051'
infile = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/郝海萍-激素检测报告-2022-05-20.xlsx'
# result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
result_info , dct = get_excel_info(infile,hospSampleID,flag='5')

data1 = set_data1(dct)
inpdf =  'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/郝海萍-激素检测报告-2022-05-20.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 14:45:17 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '2342', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;2343127051&lt;/lis_Barcode&gt;&lt;sample_no&gt;1&lt;/sample_no&gt;&lt;pat_no&gt;2001909006&lt;/pat_no&gt;&lt;pat_id&gt;2001909006&lt;/pat_id&gt;&lt;pat_name&gt;郝海萍&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-18 08:45:38&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1999-07-05 00:00:00&lt;/pat_birthday&gt;&lt;pat

In [54]:
hospSampleID = '2443127052'
infile = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/郝海萍-激素检测报告-2022-05-20.xlsx'
# result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
result_info , dct = get_excel_info(infile,hospSampleID,flag='17')

data1 = set_data1(dct)
inpdf =  'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/郝海萍-激素检测报告-2022-05-20.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 14:45:24 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '2322', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;2443127052&lt;/lis_Barcode&gt;&lt;sample_no&gt;2443127052&lt;/sample_no&gt;&lt;pat_no&gt;2001909006&lt;/pat_no&gt;&lt;pat_id&gt;2001909006&lt;/pat_id&gt;&lt;pat_name&gt;郝海萍&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-18 08:45:41&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1999-07-05 00:00:00&lt;/pat_birthday&g

In [55]:
hospSampleID = '4043146697'
file_name = '何丽敏-激素检测报告-2022-05-25'
file_path = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/' + file_name 
infile = file_path + '.xlsx'
result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
# result_info , dct = get_excel_info(infile,hospSampleID,flag='17')
data1 = set_data1(dct)
infile = file_path + '.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 14:52:31 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '2300', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;4043146697&lt;/lis_Barcode&gt;&lt;sample_no&gt;3&lt;/sample_no&gt;&lt;pat_no&gt;2001155124&lt;/pat_no&gt;&lt;pat_id&gt;2001155124&lt;/pat_id&gt;&lt;pat_name&gt;何丽敏&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-20 08:45:47&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1998-03-14 00:00:00&lt;/pat_birthday&gt;&lt;pat

In [56]:
hospSampleID = '4143146698'
file_name = '何丽敏-激素检测报告-2022-05-25'
file_path = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/' + file_name 
infile = file_path + '.xlsx'
# result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
result_info , dct = get_excel_info(infile,hospSampleID,flag='17')
data1 = set_data1(dct)
infile = file_path + '.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 14:53:16 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1704', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;4143146698&lt;/lis_Barcode&gt;&lt;sample_no&gt;4&lt;/sample_no&gt;&lt;pat_no&gt;2001155124&lt;/pat_no&gt;&lt;pat_id&gt;2001155124&lt;/pat_id&gt;&lt;pat_name&gt;何丽敏&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-20 08:47:23&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1998-03-14 00:00:00&lt;/pat_birthday&gt;&lt;pat

In [57]:
hospSampleID = '2743136442'
file_name = '何佩怡-激素检测报告-2022-05-25'
file_path = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/' + file_name 
infile = file_path + '.xlsx'
result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
# result_info , dct = get_excel_info(infile,hospSampleID,flag='17')
data1 = set_data1(dct)
infile = file_path + '.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 14:55:19 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '2341', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;2743136442&lt;/lis_Barcode&gt;&lt;sample_no&gt;11&lt;/sample_no&gt;&lt;pat_no&gt;2001904103&lt;/pat_no&gt;&lt;pat_id&gt;2001904103&lt;/pat_id&gt;&lt;pat_name&gt;何佩怡&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-19 08:24:11&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1994-12-08 00:00:00&lt;/pat_birthday&gt;&lt;pa

In [58]:
hospSampleID = '2843136443'
file_name = '何佩怡-激素检测报告-2022-05-25'
file_path = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/' + file_name 
infile = file_path + '.xlsx'
# result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
result_info , dct = get_excel_info(infile,hospSampleID,flag='17')
data1 = set_data1(dct)
infile = file_path + '.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 14:55:25 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1745', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;2843136443&lt;/lis_Barcode&gt;&lt;sample_no&gt;12&lt;/sample_no&gt;&lt;pat_no&gt;2001904103&lt;/pat_no&gt;&lt;pat_id&gt;2001904103&lt;/pat_id&gt;&lt;pat_name&gt;何佩怡&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-19 08:24:27&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1994-12-08 00:00:00&lt;/pat_birthday&gt;&lt;pa

In [59]:
hospSampleID = '3343165806'
file_name = '黄灵怡-激素检测报告-2022-05-25'
file_path = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/' + file_name 
infile = file_path + '.xlsx'
result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
# result_info , dct = get_excel_info(infile,hospSampleID,flag='17')
data1 = set_data1(dct)
infile = file_path + '.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 14:56:28 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '2338', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;3343165806&lt;/lis_Barcode&gt;&lt;sample_no&gt;21&lt;/sample_no&gt;&lt;pat_no&gt;2000328597&lt;/pat_no&gt;&lt;pat_id&gt;2000328597&lt;/pat_id&gt;&lt;pat_name&gt;黄灵怡&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-22 09:23:36&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;2000-12-21 00:00:00&lt;/pat_birthday&gt;&lt;pa

In [60]:
hospSampleID = '3443165807'
file_name = '黄灵怡-激素检测报告-2022-05-25'
file_path = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/' + file_name 
infile = file_path + '.xlsx'
# result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
result_info , dct = get_excel_info(infile,hospSampleID,flag='17')
data1 = set_data1(dct)
infile = file_path + '.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 14:56:34 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1742', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;3443165807&lt;/lis_Barcode&gt;&lt;sample_no&gt;22&lt;/sample_no&gt;&lt;pat_no&gt;2000328597&lt;/pat_no&gt;&lt;pat_id&gt;2000328597&lt;/pat_id&gt;&lt;pat_name&gt;黄灵怡&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-22 09:23:42&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;2000-12-21 00:00:00&lt;/pat_birthday&gt;&lt;pa

In [66]:
hospSampleID = '3143146922'
file_name = '李捷-激素检测报告-2022-05-25'
file_path = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/' + file_name 
infile = file_path + '.xlsx'
result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
# result_info , dct = get_excel_info(infile,hospSampleID,flag='17')
data1 = set_data1(dct)
infile = file_path + '.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 14:59:07 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '2297', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;3143146922&lt;/lis_Barcode&gt;&lt;sample_no&gt;5&lt;/sample_no&gt;&lt;pat_no&gt;2001913978&lt;/pat_no&gt;&lt;pat_id&gt;2001913978&lt;/pat_id&gt;&lt;pat_name&gt;李捷&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-20 08:46:30&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1997-02-17 00:00:00&lt;/pat_birthday&gt;&lt;pat_

In [62]:
hospSampleID = '3243146923'
file_name = '李捷-激素检测报告-2022-05-25'
file_path = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/' + file_name 
infile = file_path + '.xlsx'
# result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
result_info , dct = get_excel_info(infile,hospSampleID,flag='17')
data1 = set_data1(dct)
infile = file_path + '.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 14:57:35 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1701', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;3243146923&lt;/lis_Barcode&gt;&lt;sample_no&gt;6&lt;/sample_no&gt;&lt;pat_no&gt;2001913978&lt;/pat_no&gt;&lt;pat_id&gt;2001913978&lt;/pat_id&gt;&lt;pat_name&gt;李捷&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-20 08:47:03&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1997-02-17 00:00:00&lt;/pat_birthday&gt;&lt;pat_

In [65]:
hospSampleID = '2243137022'
file_name = '刘晓翠-激素检测报告-2022-05-25'
file_path = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/' + file_name 
infile = file_path + '.xlsx'
result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
# result_info , dct = get_excel_info(infile,hospSampleID,flag='17')
data1 = set_data1(dct)
infile = file_path + '.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 14:58:57 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '2341', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;2243137022&lt;/lis_Barcode&gt;&lt;sample_no&gt;13&lt;/sample_no&gt;&lt;pat_no&gt;2001891777&lt;/pat_no&gt;&lt;pat_id&gt;2001891777&lt;/pat_id&gt;&lt;pat_name&gt;刘晓翠&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-19 08:57:47&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1991-10-22 00:00:00&lt;/pat_birthday&gt;&lt;pa

In [64]:
hospSampleID = '2343137023'
file_name = '刘晓翠-激素检测报告-2022-05-25'
file_path = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/' + file_name 
infile = file_path + '.xlsx'
# result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
result_info , dct = get_excel_info(infile,hospSampleID,flag='17')
data1 = set_data1(dct)
infile = file_path + '.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 14:58:38 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1745', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;2343137023&lt;/lis_Barcode&gt;&lt;sample_no&gt;14&lt;/sample_no&gt;&lt;pat_no&gt;2001891777&lt;/pat_no&gt;&lt;pat_id&gt;2001891777&lt;/pat_id&gt;&lt;pat_name&gt;刘晓翠&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-19 08:58:37&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1991-10-22 00:00:00&lt;/pat_birthday&gt;&lt;pa

In [67]:
hospSampleID = '2943156181'
file_name = '罗冬红-激素检测报告-2022-05-25'
file_path = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/' + file_name 
infile = file_path + '.xlsx'
result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
# result_info , dct = get_excel_info(infile,hospSampleID,flag='17')
data1 = set_data1(dct)
infile = file_path + '.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 15:00:00 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '2296', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;2943156181&lt;/lis_Barcode&gt;&lt;sample_no&gt;15&lt;/sample_no&gt;&lt;pat_no&gt;2001916669&lt;/pat_no&gt;&lt;pat_id&gt;2001916669&lt;/pat_id&gt;&lt;pat_name&gt;罗冬红&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-21 09:01:03&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1994-11-12 00:00:00&lt;/pat_birthday&gt;&lt;pa

In [68]:
hospSampleID = '3043156182'
file_name = '罗冬红-激素检测报告-2022-05-25'
file_path = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/' + file_name 
infile = file_path + '.xlsx'
# result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
result_info , dct = get_excel_info(infile,hospSampleID,flag='17')
data1 = set_data1(dct)
infile = file_path + '.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 15:00:07 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1700', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;3043156182&lt;/lis_Barcode&gt;&lt;sample_no&gt;16&lt;/sample_no&gt;&lt;pat_no&gt;2001916669&lt;/pat_no&gt;&lt;pat_id&gt;2001916669&lt;/pat_id&gt;&lt;pat_name&gt;罗冬红&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-22 15:57:45&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1994-11-12 00:00:00&lt;/pat_birthday&gt;&lt;pa

In [69]:
hospSampleID = '3243177523'
file_name = '潘琦-激素检测报告-2022-05-25'
file_path = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/' + file_name 
infile = file_path + '.xlsx'
result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
# result_info , dct = get_excel_info(infile,hospSampleID,flag='17')
data1 = set_data1(dct)
infile = file_path + '.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 15:01:03 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '2302', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;3243177523&lt;/lis_Barcode&gt;&lt;sample_no&gt;17&lt;/sample_no&gt;&lt;pat_no&gt;2001921127&lt;/pat_no&gt;&lt;pat_id&gt;2001921127&lt;/pat_id&gt;&lt;pat_name&gt;潘琦&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-23 10:56:18&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1996-09-03 00:00:00&lt;/pat_birthday&gt;&lt;pat

In [70]:
hospSampleID = '3343177524'
file_name = '潘琦-激素检测报告-2022-05-25'
file_path = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/' + file_name 
infile = file_path + '.xlsx'
# result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
result_info , dct = get_excel_info(infile,hospSampleID,flag='17')
data1 = set_data1(dct)
infile = file_path + '.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 15:01:09 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1706', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;3343177524&lt;/lis_Barcode&gt;&lt;sample_no&gt;18&lt;/sample_no&gt;&lt;pat_no&gt;2001921127&lt;/pat_no&gt;&lt;pat_id&gt;2001921127&lt;/pat_id&gt;&lt;pat_name&gt;潘琦&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-23 10:56:24&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1996-09-03 00:00:00&lt;/pat_birthday&gt;&lt;pat

In [72]:
hospSampleID = '2843125193'
file_name = '郑佳娜-激素检测报告-2022-05-25'
file_path = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/' + file_name 
infile = file_path + '.xlsx'
result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
# result_info , dct = get_excel_info(infile,hospSampleID,flag='17')
data1 = set_data1(dct)
infile = file_path + '.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 15:03:00 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '2352', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;2843125193&lt;/lis_Barcode&gt;&lt;sample_no&gt;7&lt;/sample_no&gt;&lt;pat_no&gt;2000196036&lt;/pat_no&gt;&lt;pat_id&gt;2000196036&lt;/pat_id&gt;&lt;pat_name&gt;郑佳娜&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-19 10:27:38&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1993-10-13 00:00:00&lt;/pat_birthday&gt;&lt;pat

In [74]:
hospSampleID = '2943125194'
file_name = '郑佳娜-激素检测报告-2022-05-25'
file_path = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/' + file_name 
infile = file_path + '.xlsx'
# result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
result_info , dct = get_excel_info(infile,hospSampleID,flag='17')
data1 = set_data1(dct)
infile = file_path + '.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 15:03:26 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1756', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;2943125194&lt;/lis_Barcode&gt;&lt;sample_no&gt;8&lt;/sample_no&gt;&lt;pat_no&gt;2000196036&lt;/pat_no&gt;&lt;pat_id&gt;2000196036&lt;/pat_id&gt;&lt;pat_name&gt;郑佳娜&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-19 10:27:48&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1993-10-13 00:00:00&lt;/pat_birthday&gt;&lt;pat

In [75]:
hospSampleID = '2843125193'
file_name = '庄洁-激素检测报告-2022-05-25'
file_path = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/' + file_name 
infile = file_path + '.xlsx'
result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
# result_info , dct = get_excel_info(infile,hospSampleID,flag='17')
data1 = set_data1(dct)
infile = file_path + '.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 15:04:09 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '2352', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;2843125193&lt;/lis_Barcode&gt;&lt;sample_no&gt;7&lt;/sample_no&gt;&lt;pat_no&gt;2000196036&lt;/pat_no&gt;&lt;pat_id&gt;2000196036&lt;/pat_id&gt;&lt;pat_name&gt;郑佳娜&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-19 10:27:38&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1993-10-13 00:00:00&lt;/pat_birthday&gt;&lt;pat

In [76]:
hospSampleID = '2943125194'
file_name = '庄洁-激素检测报告-2022-05-25'
file_path = 'E:/linuxPathFile/shenZhen_CentOs/project/long-term-proj/pcos/PCOS_transfer_data-new/数据/省妇幼报告/' + file_name 
infile = file_path + '.xlsx'
# result_info , dct = get_excel_info(infile,hospSampleID,flag='5')
result_info , dct = get_excel_info(infile,hospSampleID,flag='17')
data1 = set_data1(dct)
infile = file_path + '.pdf'
data2 = get_pdf(inpdf)
send_data(dct,data1,result_info,data2)

patient_info.header----
{'Server': 'nginx/1.19.2', 'Date': 'Thu, 26 May 2022 15:04:16 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1756', 'Connection': 'keep-alive'}
patient_info.text----
<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://www.w3.org/2003/05/soap-envelope" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><GetLisRequestResponse xmlns="http://tempuri.org/"><GetLisRequestResult>&lt;?xml version="1.0" encoding="utf-8"?&gt;&lt;Data&gt;&lt;Data_Row&gt;&lt;lis_Barcode&gt;2943125194&lt;/lis_Barcode&gt;&lt;sample_no&gt;8&lt;/sample_no&gt;&lt;pat_no&gt;2000196036&lt;/pat_no&gt;&lt;pat_id&gt;2000196036&lt;/pat_id&gt;&lt;pat_name&gt;郑佳娜&lt;/pat_name&gt;&lt;pat_bedNo&gt;&lt;/pat_bedNo&gt;&lt;spouse_info&gt;&lt;/spouse_info&gt;&lt;blood_time&gt;2022-05-19 10:27:48&lt;/blood_time&gt;&lt;pat_sex&gt;0&lt;/pat_sex&gt;&lt;pat_birthday&gt;1993-10-13 00:00:00&lt;/pat_birthday&gt;&lt;pat